In [34]:
from langchain.vectorstores import Qdrant
from qdrant_client import QdrantClient
qdrant = QdrantClient(
    url="https://995828ea-aab5-412e-9766-dd32680878b2.europe-west3-0.gcp.cloud.qdrant.io:6333",
    api_key=qdrant_api_key,
    check_compatibility=False,
    timeout=180
)
qdrant.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=768, distance=Distance.COSINE)  # BioBERT = 768
)

True

In [48]:
len(parent_ids)

1122

In [51]:
qdrant.upload_points(
    collection_name =collection_name,
    points=fifth
)

In [59]:
import json 
with open("parent_ids.json","w") as F:
    json.dump(parent_ids,F,indent=4)

In [ ]:
with 

In [61]:
len(parent_ids)

1352

In [ ]:
from groq import Groq

client = Groq()
completion = client.chat.completions.create(
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    messages=[
        {
            "role": "user",
            "content": "WHO IS CRISTIANO RONALDO"
        }
    ],
    temperature=1,
    max_completion_tokens=1024,
    top_p=1,
    stream=True,
    stop=None,
)

for chunk in completion:
    print(chunk.choices[0].delta.content or "", end="")

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Qdrant
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from qdrant_client.http.models import PointStruct
from uuid import uuid4
from dotenv import load_dotenv


import os
import json 
import re


load_dotenv()
qdrant_api_key = os.getenv("QDRANT_API_KEY")
collection_name = "gale_books"
embedding_model = HuggingFaceEmbeddings(model_name="pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb")


listofBooks = [f"gale_articles_Vol_{i}.json" for i in range(1,6)]





def MainIndexing(listOfBooks:list):
    for book  in listofBooks:
        with open(book,"r",encoding="utf-8") as f :
            content = json.load(f)
            for i in range(len(content)):
                article_title = content[i]["title"]
                article = content[i]["content"]
                # Clean the article
                cleaned_article = CleanArticle(article)
                #create_ParentChild Chunking 
                ParentChildChunking(cleaned_article,article_title)
        print(f"The article of {book} are embedded and inserted in the DB")
        print("==================================")
                


C:\Users\HP\AppData\Local\Temp\ipykernel_8296\3329933065.py:19: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb")
c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Qdrant
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from qdrant_client.http.models import PointStruct
from uuid import uuid4
from dotenv import load_dotenv

import time
import os
import json 
import re


load_dotenv()
qdrant_api_key = os.getenv("QDRANT_API_KEY")
collection_name = "gale_books"
embedding_model = HuggingFaceEmbeddings(model_name="pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb")


listofBooks = [f"gale_articles_Vol_{i}.json" for i in range(1,6)]





def MainIndexing(Book):
        with open(Book,"r",encoding="utf-8") as f :
            content = json.load(f)
            for i in range(len(content)):
                article_title = content[i]["title"]
                article = content[i]["content"]
                # Clean the article
                cleaned_article = CleanArticle(article)
                #create_ParentChild Chunking 
                ParentChildChunking(cleaned_article,article_title)
                if i%50 == 0:
                     time.sleep(30)
        print(f"The article of {Book} are embedded and inserted in the DB")
        print("==================================")
                


C:\Users\HP\AppData\Local\Temp\ipykernel_13656\729879767.py:19: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb")
c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
parent_ids = {}
def SecondAlter(book):
    points = []
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=150,chunk_overlap= 50)
    embedding_model = HuggingFaceEmbeddings(model_name="pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb")
    with open(book,"r",encoding="utf-8") as f :
            content = json.load(f)
            for i in range(len(content)):
                article_title = content[i]["title"]
                article = content[i]["content"]
                # Clean the article
                cleaned_article = CleanArticle(article)
                chunks_doc = []
                parent_id = str(uuid4())
                parent_ids[parent_id] = cleaned_article
                # Chunk the article 
                chunks = text_splitter.create_documents(
                    [cleaned_article],
                    metadatas=[{
                        "title":article_title,# must be modified
                        "parent_id": parent_id}]
                )
                chunks_doc.extend(chunks)
                # embedd the vectors 
                vectors = embedding_model.embed_documents([c.page_content for c in chunks_doc])
                # Create Point for Qdrant injection 

                for index, doc in enumerate(chunks_doc):
                    point = PointStruct(
                        id=str(uuid4()),
                        vector=vectors[index],
                        payload={
                            **doc.metadata
                        }
                    )
                    points.append(point)
    return points


In [7]:
if __name__=="__main__":
    first = SecondAlter("gale_articles_Vol_2.json")

In [39]:
if __name__=="__main__":
    second = SecondAlter("gale_articles_Vol_3.json")

In [43]:
if __name__=="__main__":
    third = SecondAlter("gale_articles_Vol_4.json")

In [47]:
if __name__=="__main__":
    fourth = SecondAlter("gale_articles_Vol_5.json")

In [50]:
if __name__=="__main__":
    fifth = SecondAlter("gale_articles_Vol_1.json")

### <B>Cleaning Articles</B>

In [2]:
import re

def CleanArticle(article):
    # Remove GALE/scan artifacts
    cleaned_article = re.sub(r"GALE ENCYCLOPEDIA OF MEDICINE.*?Page \d+", "", article)
    cleaned_article = re.sub(r"GEM\s*-\s*\d+\s*to\s*\d+\s*-\s*[A-Z]?", "", cleaned_article)
    cleaned_article = re.sub(r"Page \d+", "", cleaned_article)
    #Fix line breaks inside paragraphs
    cleaned_article = re.sub(r"(?<!\n)(?<!\n\n)\n(?![\n•A-Z])", " ", cleaned_article )
    #Normalize multiple newlines
    cleaned_article = re.sub(r"\n{2,}", "\n\n", cleaned_article)
    #Strip excessive whitespace
    cleaned_article = re.sub(r"[ \t]+", " ", cleaned_article).strip()
    cleaned_article = re.sub(r"(?:\n)?Resources\s*\n(?:.*\n)*", " ", cleaned_article, flags=re.IGNORECASE)
    #Ensure section headers are separated clearly
    cleaned_article = re.sub(r"\n([A-Z][a-z]+(?: [A-Z][a-z]+)*):?\n", r"\n\n\1\n\n", cleaned_article)

    return cleaned_article

### <b>Create Parent-Child chunking </b>

In [25]:
ParentDocs = {}
def ParentChildChunking(cleaned_article,article_title):
    chunks_doc = []
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=150,chunk_overlap= 50)
    parent_id = str(uuid4())
    ParentDocs[parent_id] = cleaned_article
    # Chunk the article 
    chunks = text_splitter.create_documents(
        [cleaned_article],
        metadatas=[{
            "title":article_title,# must be modified
            "parent_id": parent_id}]
    )
    chunks_doc.extend(chunks)
    # embedd the vectors 
    vectors = embedding_model.embed_documents([c.page_content for c in chunks_doc])
    # Create Point for Qdrant injection 
    points = []

    for index, doc in enumerate(chunks_doc):
        point = PointStruct(
            id=str(uuid4()),
            vector=vectors[index],
            payload={
                **doc.metadata
            }
        )
        points.append(point)
     # Insert Points in Qdrant collection 
    qdrant.upsert(
    collection_name=collection_name,
    points=points
    )
    

## Working with the retrieval 

### Create the prompt for getting multiple queries

In [8]:
from langchain.prompts import ChatPromptTemplate

template = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to: {question} \n
Output ( 3 queries):"""
prompt_rag_fusion = ChatPromptTemplate.from_template(template)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_queries = (
    prompt_rag_fusion 
    | ChatOpenAI(temperature=0)
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

In [ ]:
from langchain.load import dumps, loads

def reciprocal_rank_fusion(results: list[list], k=60):
    """ Reciprocal_rank_fusion that takes multiple lists of ranked documents 
        and an optional parameter k used in the RRF formula """
    
    # Initialize a dictionary to hold fused scores for each unique document
    fused_scores = {}

    # Iterate through each list of ranked documents
    for docs in results:
        # Iterate through each document in the list, with its rank (position in the list)
        for rank, doc in enumerate(docs):
            # Convert the document to a string format to use as a key (assumes documents can be serialized to JSON)
            doc_str = dumps(doc)
            # If the document is not yet in the fused_scores dictionary, add it with an initial score of 0
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # Retrieve the current score of the document, if any
            previous_score = fused_scores[doc_str]
            # Update the score of the document using the RRF formula: 1 / (rank + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # Sort the documents based on their fused scores in descending order to get the final reranked results
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # Return the reranked results as a list of tuples, each containing the document and its fused score
    return reranked_results
retrieval_chain_rag_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion

In [ ]:
def retriever_per_query(query:str,client):
    query_vector = embedding_model.embed_query(query)
    search_results = client.search(
        collection_name= collection_name,
        query_vector=query_vector,
        limit=5
    )
    return search_results
    